# start

## load the dependencies


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import os
import IPython.display as ipd
import librosa
import tensorflow as tf
#import tensorflow_io as tfio
from scipy.io import wavfile
import soundfile as sf
from IPython.display import Audio
import io
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F


## loading the data

In [4]:
# load the data its in the Train directory its not a csv its just wav files

# get every file from the Train directory
train_dir = "C:/Users/zboom/Documents/Uni/Year 3/Deep Learning/DeepLearning copy/DeepLearning copy/Train"
train_files = os.listdir(train_dir)

# make csv file out of the files 
train_df = pd.DataFrame(train_files, columns=['file'])


# create a column with the file path 
train_df['file_path'] = train_df['file'].apply(lambda x: os.path.join(train_dir, x))


# create a column with the file name without the .wav extension
train_df['file_name'] = train_df['file'].str.replace('.wav', '')

# create a column with the accent from the first numer of the file name
train_df['accent'] = train_df['file_name'].str[0]

# create a column with male or female from the second incex of the file name
train_df['gender'] = train_df['file_name'].str[1]

train_df

,file,file_path,file_name,accent,gender
0,1f_1018.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1f_1018,1,f
1,1f_1026.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1f_1026,1,f
2,1f_1031.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1f_1031,1,f
3,1f_1070.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1f_1070,1,f
4,1f_1075.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1f_1075,1,f
...,...,...,...,...,...
3161,5m_9719.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,5m_9719,5,m
3162,5m_9742.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,5m_9742,5,m
3163,5m_9771.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,5m_9771,5,m
3164,5m_9778.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,5m_9778,5,m


In [5]:
test_dir = "C:/Users/zboom/Documents/Uni/Year 3/Deep Learning/DeepLearning copy/DeepLearning copy/Test"
test_files = os.listdir(test_dir)

test_df = pd.DataFrame(test_files, columns=['file'])
test_df['file_path'] = test_df['file'].apply(lambda x: os.path.join(test_dir, x))
test_df['file_name'] = test_df['file'].str.replace('.wav', '')

# create an empty column for the accent and gender
test_df['accent'] = ''
test_df['gender'] = ''
 
test_df

,file,file_path,file_name,accent,gender
0,1035.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1035,,
1,1074.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1074,,
2,1088.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1088,,
3,1095.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1095,,
4,1099.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,1099,,
...,...,...,...,...,...
546,9949.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,9949,,
547,9954.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,9954,,
548,9959.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,9959,,
549,9974.wav,C:/Users/zboom/Documents/Uni/Year 3/Deep Learn...,9974,,


## define augmentaion functions

In [31]:
### load data and display it

def load_audio_file(file_path):
    # The longest file is 286003 ms? long
    input_length = 286003
    data = librosa.core.load(file_path, sr = 16000)[0]
    # returns a numpy array which is a audio time series, in our case with 
    # only one channel.
    data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
    return data

def plot_time_series(data):
    fig = plt.figure(figsize=(14, 8))
    plt.title('Raw wave ')
    plt.ylabel('Amplitude')
    plt.plot(np.linspace(0, 1, len(data)), data)
    plt.show()

def array_to_wav_bytes(arr, rate=16000):
    buf = io.BytesIO()
    sf.write(buf, arr, rate, format='WAV')
    buf.seek(0)
    return buf.read()  # raw WAV bytes

def add_white_noise(file):
    amp = random.randrange(1, 21) * 0.005
    # amp between 0.005 and 0.1
    # creates a normally distributed array with the length of the file 
    wn = np.random.randn(len(file))
    file_wn = file + amp*wn
    if type(file_wn) != np.ndarray:
        print('problem, add_white_noise')
    return file_wn

def data_roll(file):
    rd = random.randint(1, 25)
    hz = rd * 10,000
    # range between 10,000 and 250,000
    file_roll = np.roll(file, hz)
    if type(file_roll) != np.ndarray:
        print('problem, data_roll')
    return file_roll

def stretch(file):
    rd = random.randint(1, 3)
    # range between 1 and 3 
    data_stretch = librosa.effects.time_stretch(file, rate = rd)
    if type(data_stretch) != np.ndarray:
        print('problem, data_stretch')
    return data_stretch

def augment(data):
    functions = [add_white_noise, data_roll, stretch]
    random_number = random.randint(0, 2)
    #print('augmented')
    function = functions[random_number]
    return function(data)
    #print('didn\'t augment')
    
def frequency_mask(data):
    spectro = data[0]
    range = random.randint(0, 30)
    band = range = random.randint(0, len(spectro))
    spectro[band: band+range] = 0
    return data

def temporal_mask(data):
    spectro = data[0]
    range = random.randint(0, 2000)
    band =  random.randint(0, 15053)
    print(band, range)
    spectro[:, band: band+range] = 0
    return data

## apply initial augmentation to the dataset

In [7]:
train_data = [] # shape is 3100 entries of [loaded file, accent]

for name, accent in zip(train_df['file_path'], train_df['accent']):
    scale, sr = librosa.load(name)
    if type(scale) != np.ndarray:
        print('problem') 
    train_data.append([scale, accent])

In [ ]:
for i in range(3):
    
    for i in range(len(train_df)):
        random_number = random.randint(0, 1)
        if random_number == 0:
            augmented_data = augment(train_data[i][0])
            train_data.append([augmented_data, train_data[i][1]])


## pad data

In [7]:
for data in train_data:
    data[0] = np.pad(data[0], (0, max(0, 286003 - len(data[0]))), "constant")

## spectrogram augmentation

In [8]:
spectro_accent = []

for i in range(100):
    print(i)
    mel_spectrogram = librosa.feature.melspectrogram(y= train_data[i][0],sr= 16000, hop_length=5)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    spectro_accent.append([log_mel_spectrogram, train_data[i][1]])

librosa.display.specshow(spectro_accent[0][0], x_axis="time", y_axis="mel", sr=sr)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


KeyboardInterrupt: 

In [24]:
spectro_accent[0][0][:, :5]

array([[  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [-53.123505, -53.123505, -53.123505, -53.123505, -53.123505],
       [-53.123505, -53.123505, -53.123505, -53.123505, -53.123505],
       [-53.123505, -53.123505, -53.123505, -53.123505, -53.123505],
       [-53.123505, -53.123505, -53.123505, -53.123505, -53.123505],
       [-53.123505, -53.123505, -5

In [27]:
def frequency_mask(data):
    spectro = data[0]
    range = random.randint(0, 30)
    band = range = random.randint(0, len(spectro))
    spectro[band: band+range] = 0
    return data

def temporal_mask(data):
    spectro = data[0]
    range = random.randint(0, 2000)
    band =  random.randint(0, 15053)
    print(band, range)
    spectro[:, band: band+range] = 0
    return data

9013 487


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## build the model

In [10]:
class CustomImageDataset(Dataset):
    def __init__(self, data, transform=None, target_transform=None):
        self.data = data
        self.transform = transform
        self.target_transforms = target_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        spectro = data[0]
        label = data[1]
        return spectro, label

In [11]:
trainset = CustomImageDataset(spectro_accent)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        print('entering first conv')
        x = self.pool(F.relu(self.conv1(x)))
        print('entering second conv')
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        print('entering first relu')
        x = F.relu(self.fc1(x))
        print('entering second relu')
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        print('finished one pass')
        return x

net = Net()

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

## failed attempts

In [ ]:
length= 0
name = None
for file in test_df['file_path']:
    data = librosa.core.load(file, sr = 16000)[0]
    if len(data) > length:
        length = len(data)
        name = file

display(ipd.Audio(name, rate=16000))

data1 = load_audio_file(name)

In [ ]:
list_spectros = []

for name in test_df['file_path']:
    sample_rate, samples = wavfile.read(name)
    frame_size = 2048
    hop_size = 512
    frames = np.array([samples[i:i+frame_size].astype(np.float64) for i in range(0, len(samples) - frame_size, hop_size)])

    window = np.hanning(frame_size)
    frames *= window

    spectrogram = np.abs(np.fft.rfft(frames, n=frame_size))

    # Compute the power spectrum
    power_spectrum = spectrogram ** 2

    # Compute the logarithmic scale spectrogram
    log_spectrogram = 10 * np.log10(power_spectrum)

    list_spectros.append(log_spectrogram)